### EXERCICES

génération à partir de chatgpt...


Exercice1 : un notebook python où vous arrivez à générer/visualiser une trajectoire.

Exercice2 : définir / implémenter des métriques statistiques et des tests d'identification des browniens.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [13]:
# pour simuler normale, on a box muller, marsaglia, ziggurat, etc. 
# mais on peut utiliser la fonction de numpy standatd_normal qui est optimisée et rapide

In [12]:
def simulate_brownian(mu=0.0, sigma=1.0, T=1.0, n=252, x0=0.0, seed=None):
    """
    Simule X_t = x0 + mu t + sigma W_t sur [0, T] avec n pas.
    Retourne t(n+1,), X(n+1,), dt.
    """
    rng = np.random.default_rng(seed)
    dt = T / n
    dX = mu * dt + sigma * np.sqrt(dt) * rng.standard_normal(n)
    X = np.empty(n + 1)
    X[0] = x0
    X[1:] = x0 + np.cumsum(dX)
    t = np.linspace(0.0, T, n + 1)
    return t, X, dt


## on doit vérifier les 4 propositions de la définition donnée dans le polycopié sur les méthodes monte carlo:

#### Un processus stochastique W = (Wt)t∈R+ est un mouvement brownien standard si, et seulement si,

### (1) W est issu de 0, i.e., W0 = 0 p.s.,

### (2) W est à trajectoires continues,

### (3) W est à accroissements indépendants, i.e., pour toutes séquences 0 = t0 < t1 < ... < tn, les variables # Wt2−Wt1, ..., #Wtn−Wtn−1 # sont indépendantes,

### (4) W est à accroissements stationnaires,i.e., pour tous s < t la valeur de Wt −Ws ne dépend que de la valeur de t − s et Wt −Ws ∼ N (0,t − s).

In [ ]:
## pour le premier point, on peut faire une vérification brute que W0 = 0 p.s. 
# en regardant les valeurs de W0 dans la simulation et en calculant leur maximum absolu. 
# On devrait trouver que ce maximum est très proche de zéro, ce qui confirme 
# que W0 est effectivement égal à zéro presque sûrement dans la simulation.

def test_W0_is_zero(W):
    print("\n=== (1) Test W0 = 0 p.s. ===")
    W0 = W[:, 0]
    max_abs = np.max(np.abs(W0))
    print(f"[A] max |W0| = {max_abs:.3e} (devrait être 0 en simulation)")

In [14]:
# on pourrait aussi faire le test de lilliefors, de shapiro wilk et de anderson darling pour la normalité des incréments

In [17]:
def simulate_bm_paths(m=2000, n=2000, T=1.0, seed=123):
    """
    Simule m trajectoires d'un Brownien standard sur [0,T] avec n pas.
    Retourne:
      t: (n+1,)
      W: (m, n+1)
      dW: (m, n)  (accroissements)
      dt
    """
    rng = np.random.default_rng(seed)
    dt = T / n
    dW = np.sqrt(dt) * rng.standard_normal(size=(m, n))
    W = np.zeros((m, n + 1), dtype=float)
    W[:, 1:] = np.cumsum(dW, axis=1)
    t = np.linspace(0.0, T, n + 1)
    return t, W, dW, dt

In [20]:
## pour le second point, on peut faire plusieurs diagnostics de continuité approchés, par exemple:
## - regarder la distribution des incréments dW et vérifier qu'ils sont bien petits et tendent vers 0 quand dt -> 0
## - calculer le "jump ratio" = max(dW^2) / sum(dW^2) pour chaque trajectoire, qui doit tendre vers 0 quand n augmente
## - faire un contrôle (grossier) de "pas de saut" via un seuil sur les incréments, par exemple en vérifiant que 
# la fraction d'incréments |dW| > 6*sqrt(dt) est très faible, ce qui serait anormal pour un processus continu

def continuity_diagnostics(m=2000, n=2000, T=1.0, seed=123):
    print("\n=== (2) Diagnostics de continuité (approchés) ===")

    # Méthode A: taille des incréments vs raffinement du pas
    _, _, dW1, dt1 = simulate_bm_paths(m=m, n=n, T=T, seed=seed)
    _, _, dW2, dt2 = simulate_bm_paths(m=m, n=4 * n, T=T, seed=seed + 1)

    max_inc1 = np.max(np.abs(dW1), axis=1)  # par trajectoire
    max_inc2 = np.max(np.abs(dW2), axis=1)

    q = [0.5, 0.9, 0.99]
    print("[A] Quantiles de max|dW| par trajectoire (doit diminuer quand dt diminue):")
    for qq in q:
        print(f"    q={qq:.2f}: dt={dt1:.2e} -> {np.quantile(max_inc1, qq):.4f} | "
              f"dt={dt2:.2e} -> {np.quantile(max_inc2, qq):.4f}")

    # Méthode B: "jump ratio" = max(dW^2) / sum(dW^2)
    # Pour un processus continu, ce ratio tend vers 0 quand le maillage se raffine.
    ratio1 = np.max(dW1**2, axis=1) / np.sum(dW1**2, axis=1)
    ratio2 = np.max(dW2**2, axis=1) / np.sum(dW2**2, axis=1)
    print("[B] Quantiles de max(dW^2)/sum(dW^2) (tend vers 0 quand n augmente):")
    for qq in q:
        print(f"    q={qq:.2f}: n={n} -> {np.quantile(ratio1, qq):.4e} | "
              f"n={4*n} -> {np.quantile(ratio2, qq):.4e}")

    # Méthode C: contrôle (grossier) de "pas de saut" via seuil
    # si un processus avait des sauts, on verrait des incréments anormalement grands.
    thr = 6.0 * np.sqrt(dt1)  # seuil ~ 6 sigma au pas dt1, ce qui est assez fin déjà!!
    frac_big = np.mean(np.abs(dW1) > thr)
    print(f"[C] Fraction d'incréments |dW| > 6*sqrt(dt) : {frac_big:.3e} (devrait être très faible)")

In [22]:
## pour le troisième point, pour vérifier l'indépendance des accroissements, on peut faire plusieurs tests:
## - calculer la corrélation lag-1 (Pearson) sur tout le panel d'incréments, qui doit être proche de 0
## - faire un test de Ljung-Box sur une trajectoire pour vérifier l'absence d'autocorrélation
## - faire un test par permutation sur la corrélation pour vérifier que la corrélation observée est significative ou pas
## - vérifier l'indépendance entre blocs disjoints d'incréments, 
# par exemple en calculant la corrélation entre la somme des incréments sur [0,T/2] et la somme sur [T/2,T], 
# qui doit être proche de 0 pour un processus à accroissements indépendants.

def independence_tests(dW, max_lag=20, seed=0):
    print("\n=== (3) Tests d'indépendance des accroissements ===")

    # A) Corrélation lag-1 (Pearson) sur tout le panel
    x = dW[:, :-1].ravel()
    y = dW[:, 1:].ravel()
    corr = np.corrcoef(x, y)[0, 1]
    print(f"[A] Corr(dW_t, dW_{'{'}t+dt{'}'}) ≈ {corr:.4f} (devrait être ~0)")

    # B) Ljung-Box sur une trajectoire (test d'absence d'autocorr)
    from statsmodels.stats.diagnostic import acorr_ljungbox
    if acorr_ljungbox is not None:
        series = dW[0]  # 1 trajectoire
        L = min(max_lag, len(series) // 5 if len(series) >= 5 else 1)
        lb = acorr_ljungbox(series, lags=L, return_df=True)
        p_last = float(lb["lb_pvalue"].iloc[-1])
        print(f"[B] Ljung-Box (traj 0) jusqu'au lag {L}: p-value={p_last:.3g} (grand => pas d'autocorr détectée)")
    else:
        print("[B] statsmodels non dispo -> Ljung-Box ignoré")

    # C) Test par permutation sur corrélation (robuste, ne suppose pas la normalité)
    rng = np.random.default_rng(seed)
    idx = rng.permutation(len(y))
    corr_perm = np.corrcoef(x, y[idx])[0, 1]
    print(f"[C] Corr après permutation (doit ressembler à 0): {corr_perm:.4f}")

    # D) Indépendance entre blocs disjoints: somme des incréments sur [0,T/2] vs [T/2,T]
    mid = dW.shape[1] // 2
    A = dW[:, :mid].sum(axis=1)
    B = dW[:, mid:].sum(axis=1)
    corr_blocks = np.corrcoef(A, B)[0, 1]
    print(f"[D] Corr( sum dW bloc1 , sum dW bloc2 ) ≈ {corr_blocks:.4f} (devrait être ~0)")

In [ ]:
## pour le quatrième point, pour vérifier la normalité des accroissements, on peut faire plusieurs tests:
## - Kolmogorov-Smirnov vs N(0,1) sur les incréments standardisés Z = (W_{t+k}-W_t)/sqrt(k*dt) pour plusieurs k
## - faire un histogramme des incréments et le comparer à la densité d'une normale
## - faire un Q-Q plot des incréments vs une normale
## - faire un test de normalité formel (Shapiro-Wilk, Anderson-Darling, Lilliefors, etc.) sur un échantillon d'incréments

def stationary_and_gaussian_tests(W, dt, ks=(1, 2, 5, 10, 25, 50)):
    print("\n=== (4) Tests: stationnarité des accroissements + N(0, t-s) ===")
    m, n1 = W.shape[0], W.shape[1] - 1

    # Méthode A: pour chaque k, standardiser les incréments: Z = (W_{t+k}-W_t)/sqrt(k*dt) ~ N(0,1)
    print("[A] Tests sur Z = incréments standardisés (moyenne 0, variance 1, normalité):")
    for k in ks:
        if k >= n1:
            continue
        inc = W[:, k:] - W[:, :-k]               # (m, n1-k+1)
        Z = (inc / np.sqrt(k * dt)).ravel()      # pool

        meanZ = Z.mean()
        varZ = Z.var(ddof=0)

        line = f"    k={k:>3d} (tau={k*dt:.4f}): mean={meanZ:+.3e}, var={varZ:.4f}"

        # Normalité: KS vs N(0,1)
        if stats is not None:
            ks_stat, ks_p = stats.kstest(Z, "norm")  # compare à N(0,1)
            line += f", KS p={ks_p:.3g}"
        print(line)

    # Méthode B: stationnarité "dans le temps" (même k) :
    # comparer les incréments de début vs fin via KS (égalité de distribution)
    if stats is not None:
        k = min(10, n1 // 4) if n1 >= 40 else 1
        inc = (W[:, k:] - W[:, :-k])  # (m, n1-k+1)
        L = inc.shape[1]
        early = inc[:, : L // 4].ravel()
        late  = inc[:, -L // 4 :].ravel()

        # On compare distributions (KS) et variances (Levene)
        ks_stat, ks_p = stats.ks_2samp(early, late)
        lev_stat, lev_p = stats.levene(early, late, center="median")
        print(f"\n[B] Stationnarité temporelle (k={k}, tau={k*dt:.4f}):")
        print(f"    KS(early vs late) p={ks_p:.3g} (grand => pas de diff détectée)")
        print(f"    Levene(variances) p={lev_p:.3g} (grand => variances similaires)")

    else:
        print("\n[B] SciPy non dispo -> KS/Levene ignorés")

    # Méthode C: variance-temps Var(W_{t+tau}-W_t) ≈ tau (ici sigma=1)
    # Régression log(var) vs log(tau) -> pente ~1
    taus = []
    vars_tau = []
    for k in ks:
        if k >= n1:
            continue
        inc = (W[:, k:] - W[:, :-k]).ravel()
        taus.append(k * dt)
        vars_tau.append(np.var(inc, ddof=0))

    taus = np.asarray(taus, dtype=float)
    vars_tau = np.asarray(vars_tau, dtype=float)
    slope, intercept = np.polyfit(np.log(taus), np.log(vars_tau), 1)

    print("\n[C] Scaling Var(incréments) ~ tau :")
    print(f"    pente log-log ≈ {slope:.4f} (devrait être ~1 pour Brownien)")
    print("    quelques points (tau, var):")
    for tau, vv in zip(taus[:6], vars_tau[:6]):
        print(f"      tau={tau:.4f}, var={vv:.4f}")
    
    # Méthode D: Q-Q plot des incréments vs N(0, k*dt) pour un k donné
    k = min(10, n1 // 4) if n1 >= 40 else 1
    inc = (W[:, k:] - W[:, :-k]).ravel()    
    plt.figure(figsize=(6, 6))
    stats.probplot(inc, dist="norm", plot=plt)
    plt.title(f"Q-Q plot des incréments pour k={k} (tau={k*dt:.4f})")
    plt.show()

    # Méthode E: test de normalité formel sur les incréments pour un k donné 
    from statsmodels.stats.diagnostic import lilliefors
    stat, p = lilliefors(inc, dist="norm")  # normalité avec paramètres estimés
    print(f"Test de Lilliefors: stat={stat:.4f}, p={p:.4f}")


In [24]:
def main():
    # paramètres
    m = 2000   # nb de trajectoires
    n = 2000   # pas de temps
    T = 1.0
    seed = 123

    t, W, dW, dt = simulate_bm_paths(m=m, n=n, T=T, seed=seed)

    # (1)
    test_W0_is_zero(W)

    # (2) plusieurs diagnostics (raffinement + ratios)
    continuity_diagnostics(m=m, n=n, T=T, seed=seed)

    # (3)
    independence_tests(dW, max_lag=20, seed=seed)

    # (4)
    stationary_and_gaussian_tests(W, dt, ks=(1, 2, 5, 10, 25, 50, 100, 200))